<a href="https://colab.research.google.com/github/Saba-Gul/End-to-End-RAG-App-with-Groq-and-Qdrant/blob/main/Groq_End_to_End_App_with_RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **End-to-End RAG App with Groq and Qdrant**

In [1]:
!pip -q install langchain openai tiktoken qdrant-client langchain_openai  pypdf langchainhub  langchain_community langchain-groq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 929.1 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 990.6/990.6 kB 16.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 360.7/360.7 kB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 33.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 254.1/254.1 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.8/295.8 kB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 39.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.5/103.5 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 47.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.7/5.7 MB 58.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/7

In [2]:
!pip -q install langchain openai tiktoken qdrant-client langchain_openai  pypdf langchainhub  langchain_community langchain-groq langchain-huggingface

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 3.6 MB/s eta 0:00:00


In [3]:
from langchain_groq import ChatGroq
from langchain import hub
import tiktoken
from langchain_community.vectorstores import Qdrant
from qdrant_client import QdrantClient
from google.colab import userdata
from qdrant_client import QdrantClient
from langchain_openai import OpenAIEmbeddings
from langchain_openai import ChatOpenAI
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.llms import OpenAI
from langchain.document_loaders import  PyPDFLoader
from langchain.document_loaders import DirectoryLoader
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain.prompts import PromptTemplate

Qdrant is an open-source vector search engine designed to handle and query high-dimensional vector data efficiently. It is used to perform similarity searches on large sets of data, enabling applications like semantic search, recommendation systems, and more. Qdrant works by indexing vector embeddings, which are numerical representations of data (such as text chunks), allowing fast retrieval of the most relevant results based on vector similarity.

- **Create a qdrant account and retrieve the url and API key**
- **for using open-source LLMs, we will using Groq API key**

In [4]:
qdrant_url=userdata.get('quadrant_url')
qdrant_api_key=userdata.get('quadrant_API')
# openai_api_key=userdata.get('OPENAI_API_KEY')
groq_api_key=userdata.get('GROK-API-Key')


In [6]:
from langchain_community.document_loaders import PyPDFLoader

loader=PyPDFLoader("/content/thousandslpendidsuns.pdf")
documents=loader.load()

In [7]:
documents

[Document(metadata={'source': '/content/thousandslpendidsuns.pdf', 'page': 0}, page_content='A THOUSAND SPLENDID SUNS             \n1 \n  \n \n \n \n \n \n \n \n \n \n \n \n \n \nTitle: A Thousand Splendid Suns  \nAuthor: Khaled Hosseini  \nEdition: IV  \nLanguage: English  \nDate first posted: July 2007  \nDate  recently updated: Jan 2008  \n \nThis eBook was produ ced by: Larry Zhao in Beijing  '),
 Document(metadata={'source': '/content/thousandslpendidsuns.pdf', 'page': 1}, page_content='A THOUSAND SPLENDID SUNS             \n2 \n  \nA THOUSAND  SPLENDID  SUNS \n \nKHALED HOSSEINI  \n \n \n \n \n \nA T HOUSAND  SPLENDID  SUNS \n \nRIVEKHEAD BOOKS  \n \nA member of Penguin Group (USA) Inc. New York  \n \n2007  \n \nFrom the Library of Nicolas Naprstek  \n \n \n \n \n '),
 Document(metadata={'source': '/content/thousandslpendidsuns.pdf', 'page': 2}, page_content='A THOUSAND SPLENDID SUNS             \n3 \n RIVERHEAD BOOKS  \n \nPublished by the Penguin Group  \nPenguin Group (USA) In

In [8]:
documents[:5]

[Document(metadata={'source': '/content/thousandslpendidsuns.pdf', 'page': 0}, page_content='A THOUSAND SPLENDID SUNS             \n1 \n  \n \n \n \n \n \n \n \n \n \n \n \n \n \nTitle: A Thousand Splendid Suns  \nAuthor: Khaled Hosseini  \nEdition: IV  \nLanguage: English  \nDate first posted: July 2007  \nDate  recently updated: Jan 2008  \n \nThis eBook was produ ced by: Larry Zhao in Beijing  '),
 Document(metadata={'source': '/content/thousandslpendidsuns.pdf', 'page': 1}, page_content='A THOUSAND SPLENDID SUNS             \n2 \n  \nA THOUSAND  SPLENDID  SUNS \n \nKHALED HOSSEINI  \n \n \n \n \n \nA T HOUSAND  SPLENDID  SUNS \n \nRIVEKHEAD BOOKS  \n \nA member of Penguin Group (USA) Inc. New York  \n \n2007  \n \nFrom the Library of Nicolas Naprstek  \n \n \n \n \n '),
 Document(metadata={'source': '/content/thousandslpendidsuns.pdf', 'page': 2}, page_content='A THOUSAND SPLENDID SUNS             \n3 \n RIVERHEAD BOOKS  \n \nPublished by the Penguin Group  \nPenguin Group (USA) In

In [9]:
len(documents)

253

In [10]:
text_splitter=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200)
texts=text_splitter.split_documents(documents)

In [11]:
texts[:5]

[Document(metadata={'source': '/content/thousandslpendidsuns.pdf', 'page': 0}, page_content='A THOUSAND SPLENDID SUNS             \n1 \n  \n \n \n \n \n \n \n \n \n \n \n \n \n \nTitle: A Thousand Splendid Suns  \nAuthor: Khaled Hosseini  \nEdition: IV  \nLanguage: English  \nDate first posted: July 2007  \nDate  recently updated: Jan 2008  \n \nThis eBook was produ ced by: Larry Zhao in Beijing'),
 Document(metadata={'source': '/content/thousandslpendidsuns.pdf', 'page': 1}, page_content='A THOUSAND SPLENDID SUNS             \n2 \n  \nA THOUSAND  SPLENDID  SUNS \n \nKHALED HOSSEINI  \n \n \n \n \n \nA T HOUSAND  SPLENDID  SUNS \n \nRIVEKHEAD BOOKS  \n \nA member of Penguin Group (USA) Inc. New York  \n \n2007  \n \nFrom the Library of Nicolas Naprstek'),
 Document(metadata={'source': '/content/thousandslpendidsuns.pdf', 'page': 2}, page_content="A THOUSAND SPLENDID SUNS             \n3 \n RIVERHEAD BOOKS  \n \nPublished by the Penguin Group  \nPenguin Group (USA) Inc., 375 Hudson Stre

In [12]:
len(texts)

842

In [13]:
# For open source GROQ
from langchain_huggingface import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
qdrant=Qdrant.from_documents(
    texts,
    embedding= embeddings,
    url=qdrant_url,
    prefer_grpc=True,
    api_key=qdrant_api_key,
    collection_name="llm-app",
    force_recreate=True
)


/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [15]:
qdrant=Qdrant.from_documents(
    texts,
    embedding= embeddings,
    url=qdrant_url,
    prefer_grpc=True,
    api_key=qdrant_api_key,
    collection_name="llm-app"
)

In [16]:
qdrant_client = QdrantClient(url=qdrant_url, prefer_grpc=True, api_key=qdrant_api_key)

In [17]:
retriever=qdrant.as_retriever(search_kwargs={"k": 5})

In [18]:
query = "Who was Marium?"
docs=retriever.get_relevant_documents(query)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


In [19]:
#for better print of the docs
def pretty_print_docs(docs):
    print(
        f"\n{'-' * 100}\n".join(
            [f"Document {i+1}:\n\n" + d.page_content for i, d in enumerate(docs)]
        )
    )

In [20]:
docs

[Document(metadata={'page': 4, 'source': '/content/thousandslpendidsuns.pdf', '_id': 'acbc7a63-d20a-4e53-9138-465183cf495d', '_collection_name': 'llm-app'}, page_content='people had, things such as love, family, home, acceptance.  \n \n  Jalil never called Mariam this name. Jalil said she was his little flower. He was fond of \nsitting her on his lap and telling her stories, like the time he told her that Herat, the city \nwhere Mari am was bo rn, in 1959, had once been the cradle of Persian culture, the home of \nwriters, painters, and Sufis.  \n \n  "You couldn\'t stretch a leg here without poking a poet in the ass," he laughed.  M'),
 Document(metadata={'page': 4, 'source': '/content/thousandslpendidsuns.pdf', '_id': '20066943-1c18-4f68-b427-a7a2c0e0e158', '_collection_name': 'llm-app'}, page_content='people had, things such as love, family, home, acceptance.  \n \n  Jalil never called Mariam this name. Jalil said she was his little flower. He was fond of \nsitting her on his lap an

In [21]:
docs[0].metadata

{'page': 4,
 'source': '/content/thousandslpendidsuns.pdf',
 '_id': 'acbc7a63-d20a-4e53-9138-465183cf495d',
 '_collection_name': 'llm-app'}

In [22]:
docs[0].page_content

'people had, things such as love, family, home, acceptance.  \n \n  Jalil never called Mariam this name. Jalil said she was his little flower. He was fond of \nsitting her on his lap and telling her stories, like the time he told her that Herat, the city \nwhere Mari am was bo rn, in 1959, had once been the cradle of Persian culture, the home of \nwriters, painters, and Sufis.  \n \n  "You couldn\'t stretch a leg here without poking a poet in the ass," he laughed.  M'

In [23]:
pretty_print_docs(docs)

Document 1:

people had, things such as love, family, home, acceptance.  
 
  Jalil never called Mariam this name. Jalil said she was his little flower. He was fond of 
sitting her on his lap and telling her stories, like the time he told her that Herat, the city 
where Mari am was bo rn, in 1959, had once been the cradle of Persian culture, the home of 
writers, painters, and Sufis.  
 
  "You couldn't stretch a leg here without poking a poet in the ass," he laughed.  M
----------------------------------------------------------------------------------------------------
Document 2:

people had, things such as love, family, home, acceptance.  
 
  Jalil never called Mariam this name. Jalil said she was his little flower. He was fond of 
sitting her on his lap and telling her stories, like the time he told her that Herat, the city 
where Mari am was bo rn, in 1959, had once been the cradle of Persian culture, the home of 
writers, painters, and Sufis.  
 
  "You couldn't stretch a leg he

In [24]:
docs[0].metadata.items()

dict_items([('page', 4), ('source', '/content/thousandslpendidsuns.pdf'), ('_id', 'acbc7a63-d20a-4e53-9138-465183cf495d'), ('_collection_name', 'llm-app')])

In [28]:
retriever=qdrant.as_retriever(search_kwargs={"k": 10})

prompt=PromptTemplate(

template="""
# Your role
You are a brilliant expert at understanding the intent of the questioner and the crux of the question, and providing the most optimal answer  from the docs to the questioner's needs from the documents you are given.


# Instruction
Your task is to answer the question  using the following pieces of retrieved context delimited by XML tags.

<retrieved context>
Retrieved Context:
{context}
</retrieved context>


# Constraint
1. Think deeply and multiple times about the user's question\nUser's question:\n{question}\nYou must understand the intent of their question and provide the most appropriate answer.
- Ask yourself why to understand the context of the question and why the questioner asked it, reflect on it, and provide an appropriate response based on what you understand.
2. Choose the most relevant content(the key content that directly relates to the question) from the retrieved context and use it to generate an answer.
3. Generate a concise, logical answer. When generating the answer, Do Not just list your selections, But rearrange them in context so that they become paragraphs with a natural flow.
4. When you don't have retrieved context for the question or If you have a retrieved documents, but their content is irrelevant to the question, you should answer 'I can't find the answer to that question in the material I have'.
5. Use five sentences maximum. Keep the answer concise but logical/natural/in-depth.
6. At the end of the response provide metadata provided in the relevant docs , For example:"Metadata: page: 19, source: /content/OCR_RSCA/Analyse docs JVB + mails et convention FOOT INNOVATION.pdf'. Return Just the page and source


# Question:
{question}""",
    input_variables=["context","question"]
)
#llm = ChatOpenAI(model_name="gpt-4o-mini", temperature=0,openai_api_key=openai_api_key)
groq_llm= ChatGroq(
    model="llama3-8b-8192",
    temperature=0,
    groq_api_key=groq_api_key,
    max_retries=2,
)


def format_docs(docs):
    formatted_docs = []
    for doc in docs:
        # Format the metadata into a string
        metadata_str = ', '.join(f"{key}: {value}" for key, value in doc.metadata.items())

        # Combine page content with its metadata
        doc_str = f"{doc.page_content}\nMetadata: {metadata_str}"

        # Append to the list of formatted documents
        formatted_docs.append(doc_str)

    # Join all formatted documents with double newlines
    return "\n\n".join(formatted_docs)




In [29]:
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | groq_llm
    | StrOutputParser()
)

In [30]:
rag_chain.invoke("Who was Marium?")

"Mariam was a woman who had been born out of wedlock, a harami child, and had lived a life marked by hardship and struggle. Despite her difficult circumstances, she had found love and been loved back, and had become a mother and a person of consequence. She had also learned to uproot the poisonous flowers of love and hope, which had damaged her in the past. Mariam's story is one of resilience and the human capacity to find meaning and purpose in the face of adversity.\n\nMetadata: page: 153, source: /content/thousandslpendidsuns.pdf"

In [31]:
rag_chain.invoke("Summarize the key points of the document")

'After carefully considering the user\'s question, I understand that they are seeking a concise summary of the key points from the document "A Thousand Splendid Suns" by Khaled Hosseini.\n\nFrom the retrieved context, I have identified the most relevant content that directly relates to the question. Here is a summary of the key points:\n\nThe story revolves around the lives of two Afghan women, Mariam and Laila, who are bound together by their experiences and struggles. The novel explores themes of war, violence, and the oppression of women in Afghanistan. Mariam, an illegitimate child, is forced to live in a kolba (a small, makeshift hut) and later ends up in a women\'s prison. Laila, on the other hand, is a young girl who loses her family and is forced to marry a Taliban soldier. The two women meet and form a bond, finding solace and support in each other.\n\nThe novel also highlights the struggles of the Afghan people during the Soviet occupation and the subsequent Taliban regime. T

In [33]:
rag_chain.invoke("Who was Laila?")

"Laila is the protagonist of the story, a young girl who lives in Afghanistan during the war. She is a bright and curious student who is forced to grow up quickly due to the harsh realities of war. Laila's life is marked by loss and hardship, but she also finds moments of beauty and joy in her relationships with her family and friends.\n\nAs the story progresses, Laila's character evolves, and she becomes more aware of the world around her. She begins to sense the danger and uncertainty that comes with living in a war-torn country, and she starts to make decisions that will shape her future.\n\nThroughout the story, Laila's emotions and thoughts are deeply intertwined with her experiences, and her character is developed through her interactions with others. Her story is a powerful exploration of the human spirit and its ability to persevere in the face of adversity.\n\nMetadata: page: 92, source: /content/thousandslpendidsuns.pdf"